In [ ]:
import json#format texte dérivé de la notation objets de JavaScript, données structurées en paires clés/valeurs


*Fonction* Pour Lire le CoNLLu

In [ ]:
def lire_conllu(fichier):#fonction pour analyser le texte au format coNLL-U

    fichier_input=open(fichier,mode='r',encoding='utf8')#ouvre le fichier en mode lecture avec encodage UTF-8
    contenu=fichier_input.readlines()#lit toutes les lignes du fichier et les stocke dans une liste
    fichier_input.close()#ferme le fichier pour libérer les ressources

    phrases = []#liste vide pour stocker les phrases analysées
    phrase_actuelle = []#liste temporaire pour stocker les tokens d'une phrase en cours

    for ligne in contenu:#parcourt chaque ligne du fichier
        ligne = ligne.strip()#supprime les espaces inutiles et les sauts de ligne
        if ligne.startswith("#") or ligne == "":#ignore les commentaires et les lignes vides
            if phrase_actuelle: #si une phrase en cours existe
                phrases.append(phrase_actuelle)#ajout de la phrase à la liste phrases
                phrase_actuelle = []#réinitialise phrase_actuelle pour commencer une nouvelle phrase
            continue#ignore le reste de l'itération, passe à la ligne suivante

        colonnes = ligne.split("\t")#divise la ligne en colonnes
        if len(colonnes) >= 4:#vérifie que la ligne contient 4 colonnes
            token = {#dictionnaire pour représenter un mot
                'id': colonnes[0],#identifiant, numéro de découpage des tokens
                'forme': colonnes[1],#forme de surface
                'lemme': colonnes[2],#lemme
                'classgr': colonnes[3],#partie du discours
                }
            phrase_actuelle.append(token)#ajoute le token à la liste

    if phrase_actuelle:#vérifie s'il reste une phrase non ajoutée après la boucle
        phrases.append(phrase_actuelle)#ajoute la dernière phrase à la liste des phrases
    print (str(phrases))#affiche la liste des phrases analysées

    return phrases#retourne la liste de toutes les phrases analysées






FONCTION POUR COMPTER

In [ ]:
def statistiques(phrases):#fonction pour calculer des statistiques à partir d'un ensemble de phrases, où chaque phrase = liste de tokens

    ngramMinLength = int(input ("Taille minimale des n-grammes: "))#demande à l'utilisateur la taille minimale des n-grammes
    ngramMaxLength = int(input ("Taille maximale des n-grammes: "))# demande à l'utilisateur la taille maximale des n-grammes

    nbSents = len(phrases)#calcule le nombre total de phrases
    nbToks = 0#initialise le compteur pour le nombre total de tokens
    nbForms = 0#pour le nombre de formes
    nbPuncts = 0#pour le nombre de tokens ponctuation

    totalLength = 0#initialise le total des longueurs des formes pour calculer la longueur moyenne

    noun_freq = {}#initialise un dictionnaire pour stocker la fréquence des noms
    verb_freq = {}#fréquence des verbes
    adj_freq = {}#fréquence des adjectifs
    adv_freq = {}#fréquence des adverbes
    lem_freq = {}#fréquence des lemmes
    unique_lemmes = set()#ensemble pour stocker les lemmes uniques
    ngrams = {}#initialise un dictionnaire pour stocker les n-grammes
    sauter = False#initialise une variable pour ignorer certains tokens invalides


    for phrase in phrases:#parcourt chaque phrase de la liste phrase
        for token in phrase:#parcourt chaque token de la phrase actuelle
            nbToks += 1#incrémente le compteur de tokens pour chaque token rencontré
            if token['classgr'] == 'SYM' or token['classgr'] == 'X' or sauter:#vérifie si le token est à ignorer
                sauter = False # réinitialise la variable pour ignorer les prochains tokens si nécessaire
                continue#passe au token suivant
            else:#pour tous les autres types de tokens
                if token['classgr'] == 'PUNCT':#si le token est une ponctuation
                    nbPuncts += 1#incrémente le compteur de ponctuations
                else:#sinon le token est une forme

                    if "-" not in token['id']:#vérifie que le token n'est pas un mot composé
                        nbForms += 1 #incrémente le compteur de formes
                        totalLength += len(token['forme'])#ajoute la longueur de la forme au total
                    else:
                        sauter = True#active l'indicateur pour ignorer le token
                    unique_lemmes.add(token['lemme'])#ajoute le lemme à l'ensemble des lemmes uniques
                    lem_freq[token['lemme']] = lem_freq.get(token['lemme'], 0) + 1#met à jour la fréquence du lemme

                    if token['classgr'] == 'NOUN':#si le token est un nom
                        noun_freq[token['forme']] = noun_freq.get(token['forme'], 0) + 1#met à jour la fréquence du nom
                    elif token['classgr'] == 'VERB':#si le token est un verbe
                        verb_freq[token['forme']] = verb_freq.get(token['forme'], 0) + 1#met à jour la fréquence du verbe
                    elif token['classgr'] == 'ADJ':#si le token est un adjectif
                        adj_freq[token['forme']] = adj_freq.get(token['forme'], 0) + 1#met à jour la fréquence de l'adjectif
                    elif token['classgr'] == 'ADV':#si le token est un adverbe
                        adv_freq[token['forme']] = adv_freq.get(token['forme'], 0) + 1#met à jour la fréquence de l'adverbe

    for n in range(ngramMinLength, ngramMaxLength + 1):#boucle sur chaque taille d'n-grammes
        freq_ngram = {}#initialise un dictionnaire pour stocker les fréquences des n-grammes
        for sentence in phrases:#boucle sur chaque phrase
            tokens = [token['forme']for token in sentence if token['classgr'] != 'PUNCT' and token['classgr'] != 'SYM' and token['classgr'] != 'X']#extrait les formes non ignorées
            for i in range(0,len(tokens) - n + 1):#boucle sur les n-grammes possibles dans la phrase
                ngram = tuple(tokens[i:i + n])#crée un n-gramme de taille n
                freq_ngram[ngram] = freq_ngram.get(ngram, 0) + 1#met à jour la fréquence du n-gramme
        ngrams[n] = sorted(freq_ngram.items(), key=lambda x: x[1], reverse=True)#trie les n-grammes par fréquence décroissante




    nbTypes = len(unique_lemmes)#calcule le nombre de types uniques (lemmes uniques)


# trie les noms, verbes, adjectifs, adverbes, lemmes par fréquence décroissante
    noun2freq = sorted(noun_freq.items(), key=lambda x: x[1], reverse=True)
    verb2freq = sorted(verb_freq.items(), key=lambda x: x[1], reverse=True)
    adj2freq = sorted(adj_freq.items(), key=lambda x: x[1], reverse=True)
    adv2freq = sorted(adv_freq.items(), key=lambda x: x[1], reverse=True)
    lem2freq = sorted(lem_freq.items(), key=lambda x: x[1], reverse=True)


    averageFormLength = totalLength / nbForms#calcule la longueur moyenne des formes
    averageSentLength = nbForms / nbSents#calcule la longueur moyenne des phrases




    return { #retourne les statistiques sous forme de dictionnaire
        'nbsents': nbSents,
        'nbToks': nbToks,
        'nbForms': nbForms,
        'nbPuncts': nbPuncts,
        'noun2freq': noun2freq,
        'verb2freq': verb2freq,
        'adj2freq': adj2freq,
        'adv2freq': adv2freq,
        'lem2freq': lem2freq,
        'nbTypes': len(unique_lemmes),
        'averageFormLength': averageFormLength,
        'averageSentLength': averageSentLength,
        'ngrams': ngrams
    }





Tester fonction et Générer .JSON

In [ ]:

try:

    with open("statistiques.json", "w", encoding="utf-8") as fichier:
        json.dump(statistiques(lire_conllu("source.conllu")), fichier, ensure_ascii=False)
    print("Les statistiques ont été enregistrées dans le fichier statistiques.json")


except FileNotFoundError:
    print("Le fichier source.conllu n'a pas été trouvé.")
except Exception as e:
    print(f"Une erreur s'est produite : {e}")


[[{'id': '1', 'forme': 'Clarence', 'lemme': 'Clarence', 'classgr': 'PROPN'}, {'id': '2', 'forme': 'était', 'lemme': 'être', 'classgr': 'AUX'}, {'id': '3', 'forme': 'lumineux', 'lemme': 'lumineux', 'classgr': 'ADJ'}, {'id': '4', 'forme': ',', 'lemme': ',', 'classgr': 'PUNCT'}, {'id': '5', 'forme': 'et', 'lemme': 'et', 'classgr': 'CCONJ'}, {'id': '6', 'forme': 'rester', 'lemme': 'rester', 'classgr': 'VERB'}, {'id': '7', 'forme': 'près', 'lemme': 'près', 'classgr': 'ADV'}, {'id': '8', 'forme': 'de', 'lemme': 'de', 'classgr': 'ADP'}, {'id': '9', 'forme': 'lui', 'lemme': 'lui', 'classgr': 'PRON'}, {'id': '10', 'forme': "m'", 'lemme': 'moi', 'classgr': 'PRON'}, {'id': '11', 'forme': 'offrait', 'lemme': 'offrir', 'classgr': 'VERB'}, {'id': '12', 'forme': 'de', 'lemme': 'de', 'classgr': 'ADP'}, {'id': '13', 'forme': 'la', 'lemme': 'le', 'classgr': 'DET'}, {'id': '14', 'forme': 'lumière', 'lemme': 'lumière', 'classgr': 'NOUN'}, {'id': '15', 'forme': '.', 'lemme': '.', 'classgr': 'PUNCT'}], [{'i